In [1]:
pip install opencv-python numpy ultralytics pyttsx3 cvzone SpeechRecognition


Note: you may need to restart the kernel to use updated packages.


In [2]:
import cv2
import numpy as np
from ultralytics import YOLO
import pyttsx3
import threading
import queue
import cvzone
import speech_recognition as sr

# Initialize the YOLO model and video capture
model = YOLO('yolo11n-pose.pt')
cap = cv2.VideoCapture(0)

Could not understand audio
Listening...


In [3]:
pip install -U ultralytics[track]

Note: you may need to restart the kernel to use updated packages.


In [4]:
 pip install pyaudio

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install lap

Note: you may need to restart the kernel to use updated packages.


In [4]:
import threading
import queue
import pyttsx3
import speech_recognition as sr
import numpy as np
import cv2
import cvzone

# Initialize thresholds and variables
hand_raise_counter_left = 0
hand_raise_counter_right = 0
hand_raised_left = False
hand_raised_right = False
mode = None

# Initialize text-to-speech engine
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('rate', 150)
engine.setProperty('voice', voices[1].id)
speech_queue = queue.Queue()

def speak(text):
    """
    Adds the text to the speech queue for speaking.
    """
    speech_queue.put(text)

def worker_speak():
    """
    Continuously processes the speech queue to speak the text.
    """
    while True:
        text = speech_queue.get()
        if text is None:
            break
        engine.say(text)
        engine.runAndWait()

def listen_commands():
    """
    Listens for voice commands and switches modes based on the user's input.
    """
    global mode
    recognizer = sr.Recognizer()
    mic = sr.Microphone()
    while True:
        try:
            with mic as source:
                recognizer.adjust_for_ambient_noise(source)
                print("Listening...")
                audio = recognizer.listen(source)
                commands = recognizer.recognize_google(audio).lower()
                print(commands)
                if 'hand' in commands:
                    speak("Hand raise mode activated.")
                    set_mode("hand_raise")
                elif 'stop' in commands:
                    speak("Exiting the program. Goodbye!")
                    set_mode("stop")
                    break
        except sr.UnknownValueError:
            print("Could not understand audio")
        except sr.RequestError:
            print("Speech recognition error. Check your network connection.")

def set_mode(new_mode):
    """
    Updates the global mode to the specified new_mode.
    """
    global mode, hand_raise_counter_left, hand_raise_counter_right
    mode = new_mode
    if mode == "hand_raise":
        hand_raise_counter_left = 0
        hand_raise_counter_right = 0

def calculate_angle(a, b, c):
    """
    Calculates the angle between three points (a, b, c).
    """
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

# Start threads for speaking and listening to commands
thread_speak = threading.Thread(target=worker_speak, daemon=True)
thread_speak.start()
thread_listen = threading.Thread(target=listen_commands, daemon=True)
thread_listen.start()

# Main loop for video capture and processing
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame = cv2.resize(frame, (1020, 500))
    
    # Make predictions
    result = model.track(frame)
    
    if result[0].keypoints is not None:
        keypoints = result[0].keypoints.xy.cpu().numpy()  # [[[]]]
        
        for keypoint in keypoints:  # [[]]
            if len(keypoint) > 0:  # Check for presence of keypoints
                for i, point in enumerate(keypoint):  # []
                    cx, cy = int(point[0]), int(point[1])
                    cv2.circle(frame, (cx, cy), 5, (255, 0, 0), -1)
                    cvzone.putTextRect(frame, f'{i}', (cx, cy), 1, 1)
                
                # Hand raise logic for left and right hands
                if mode == "hand_raise" and len(keypoint) > 10:
                    left_wrist = (int(keypoint[9][0]), int(keypoint[9][1]))
                    right_wrist = (int(keypoint[10][0]), int(keypoint[10][1]))
                    left_shoulder = (int(keypoint[5][0]), int(keypoint[5][1]))
                    right_shoulder = (int(keypoint[6][0]), int(keypoint[6][1]))
                    
                    # Check if the left hand is raised above the left shoulder
                    if left_wrist[1] < left_shoulder[1] and left_shoulder[1] < left_wrist[1] + 100:  # Added threshold
                        if not hand_raised_left:  # Wait for the hand to be lowered before counting again
                            hand_raised_left = True
                            hand_raise_counter_left += 1
                            speak(f'Left hand raise count: {hand_raise_counter_left}')
                    else:
                        hand_raised_left = False
                    
                    # Check if the right hand is raised above the right shoulder
                    if right_wrist[1] < right_shoulder[1] and right_shoulder[1] < right_wrist[1] + 100:  # Added threshold
                        if not hand_raised_right:  # Wait for the hand to be lowered before counting again
                            hand_raised_right = True
                            hand_raise_counter_right += 1
                            speak(f'Right hand raise count: {hand_raise_counter_right}')
                    else:
                        hand_raised_right = False
    
    if mode == 'hand_raise':
        # Display hand raise counters for left and right hands
        cvzone.putTextRect(frame, f'Left Hand Raise Counter: {int(hand_raise_counter_left)}', (50, 110), 1, 1, colorR=(0, 0, 0))
        cvzone.putTextRect(frame, f'Right Hand Raise Counter: {int(hand_raise_counter_right)}', (50, 140), 1, 1, colorR=(0, 0, 0))
        
    # Display the frame
    cv2.imshow("RGB", frame)

    # Exit on 'Esc' key press or voice "stop" command
    key = cv2.waitKey(1)
    if mode == 'stop':
        break

# Release resources
cap.release()
cv2.destroyAllWindows()



Could not understand audio
0: 320x640 1 person, 99.9ms
Speed: 0.2ms preprocess, 99.9ms inference, 0.0ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 85.3ms
Speed: 5.0ms preprocess, 85.3ms inference, 4.5ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 70.1ms
Speed: 4.5ms preprocess, 70.1ms inference, 5.1ms postprocess per image at shape (1, 3, 320, 640)

Listening...
0: 320x640 1 person, 69.9ms
Speed: 0.0ms preprocess, 69.9ms inference, 5.3ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 68.8ms
Speed: 0.0ms preprocess, 68.8ms inference, 5.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 69.7ms
Speed: 0.0ms preprocess, 69.7ms inference, 5.2ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 69.7ms
Speed: 0.0ms preprocess, 69.7ms inference, 5.4ms postprocess per image at shape (1, 3, 320, 640)

0: 320x640 1 person, 80.1ms
Speed: 0.0ms preprocess, 80.1ms inference, 0.0ms 